This is not meant as an introductory lecture - this notebook showcases some of the more high-end techniques available to advanced and experienced python users

In [ ]:
# pip install rasterstats

In [ ]:
# pip install rasterio

In [ ]:
# pip install numpy_financial

In [ ]:
# pip install loguru

In [ ]:
# pip install scikit-learn

In [ ]:
# pip install cvxpy

In [ ]:
# !git clone https://github.com/Charlesfox1/geo_training

In [20]:
# standard libs
import time
import numpy as np
from loguru import logger
from multiprocessing import Pool, cpu_count

# financial libs
import numpy_financial as npf

# graphical libs
import plotly.express as px
import folium

# geospatial libs
from shapely.geometry import Point
import pandas as pd
pd.set_option('display.precision', 1)
import geopandas as gpd
from geo_training import geo_utils
import rasterio
import rasterstats

# webscraping
import requests
from bs4 import BeautifulSoup

### MonteCarlo Simulation
- use python to construct much more advanced scenario simulations

In [ ]:
def revenue_growth_pct():
    # Draw from a normal distribution, shifted, mean of 5, s.d. of 1
    return np.random.normal(loc=5, scale=1.0)

def gross_profit_margin_pct():
    # centre around 25%
    return np.random.normal(loc=45, scale=1)

def sg_and_a_growth_pct():
    # Draw from a normal distribution, shifted, mean of 5, s.d. of 1
    return np.random.normal(loc=3, scale=1.0)

def capex_required():
    # Draw from a beta distribution, shifted, mean of 5, s.d. of 1
    x = np.random.beta(a=5, b=5)
    if x > 0.65:
        return True
    else:
        return False
    
def capex_spend():
    # spend of at least 100... could be a lot more...
    return min(np.random.pareto(a=1) * 5 + 100, 5000)

def debt_cost_pct():
    # normal distribution centred on 5%, .s.d 1%, minimum of 2%
    return round(max(np.random.normal(loc=5, scale=1), 2), 1)


def scenario(_input):
    
    V = _input[0]
    years_to_model = _input[1]
    
    # Instantiate local variables
    revenue = V['initial_revenue']
    tax_rate_pct = V['tax_rate_pct']
    debt = V['initial_debt']
    sg_and_a = V['initial_sg_and_a']
    depreciable_assets = V['initial_depreciable_assets']
    depreciation_charge = depreciable_assets * (1 / V['depreciation_lifetime_years'])
    
    res = []
    for year in range(0, years_to_model):
        
        # This year's revenue is last year's multiplied by some growth
        revenue = revenue * ((100 + revenue_growth_pct()) / 100)
        # Gross profit is some fraction of revenue
        gross_margin = gross_profit_margin_pct() / 100
        gross_profit = revenue * gross_margin
        # COGS is the delta
        cogs = revenue - gross_profit
        # Grow SG&A
        sg_and_a = sg_and_a * ((100 + sg_and_a_growth_pct()) / 100)
        # EBITDA
        ebitda = gross_profit - sg_and_a
        # Depreciate assets - straightline (fixed annual charge)
        depreciable_assets -= depreciation_charge
        # EBIT
        ebit = ebitda - depreciation_charge
        # Interest
        if year < V['years_until_refinance']:
            interest_rate_pct = V['initial_interest_pct']
        elif year == V['years_until_refinance']:
            interest_rate_pct = debt_cost_pct()
        interest_charge = debt * ((interest_rate_pct) / 100)
        # Tax
        ebt = ebit - interest_charge
        tax = max(ebt * (tax_rate_pct / 100), 0)
        # PAT
        pat = ebt - tax
        # CAPEX
        if capex_required():
            capex = capex_spend()
        else:
            capex = 0
        # FCFE - Assume no change in Working Capital, Net Borrowing for simplicity
        fcfe = ebitda - interest_charge - tax - capex
        
        # add financials to a list, store
        res.append(
            (year, revenue,cogs,gross_profit,sg_and_a,
             ebitda,depreciation_charge,ebit,interest_rate_pct,
             interest_charge,ebt,tax,pat,capex,fcfe
            )
        )
    
    # generate financials dataframe for this scenario
    df = pd.DataFrame(res, columns = 
                     ['year','revenue','cogs',
                      'gross_profit','sg_and_a','ebitda',
                      'depreciation_charge','ebit','interest_rate',
                      'interest_charge','ebt','tax','pat','capex','fcfe'])
    
    # Define cashflow stream for valuation
    cashflows = [-V['business_purchase_cost']] + list(df['fcfe'])
    
    # Summary dictionary
    summary = {'revenue_cagr': round(((revenue / V['initial_revenue']) ** (1/years_to_model)) - 1,4),
            'avg_gross_margin': round(np.mean(df['gross_profit'] / df['revenue']),4),
            'refinance_interest_rate':round(interest_rate_pct / 100,4),
            'total_capex_as_pct_of_revenue':round(df['capex'].sum() / df['revenue'].sum(),4),
            'sg_and_a_as_pct_of_revenue': round(df['sg_and_a'].sum() / df['revenue'].sum(),4),
            'IRR': round(npf.irr(cashflows),3),
            'NPV': round(npf.npv(V['discount_factor'], cashflows),1)
           }
    
    return summary, df.T

In [ ]:
# Set Common Start Variables
V = {
    'initial_revenue':650,
    'tax_rate_pct':20,
    'initial_debt':4000,
    'initial_interest_pct':2,
    'years_until_refinance':5,
    'initial_sg_and_a':45,
    'initial_depreciable_assets':550,
    'depreciation_lifetime_years':10,
    'business_purchase_cost':1000,
    'discount_factor':0.07
}

_input = (V, 10)

scenario(_input)[0]

In [ ]:
scenario(_input)[1]

In [ ]:
%%timeit
_ = scenario(_input)[0]

In [ ]:
_input = (V, 10)
scenarios = []
financial_dfs = {}
# run 10,000 scenarios
start_time = time.time()
for i in range(10000):
    output = scenario(_input)
    results = output[0]
    results['scenario_id'] = i
    scenarios.append(results)
    financial_dfs[i] = output[1]
end_time = time.time()
print(f'Time elapsed: {round(end_time - start_time, 2)}s')

In [ ]:
# Visualise the results
mdf = pd.DataFrame(scenarios)

In [ ]:
mdf

### Options
- Visualise scenarios by condition (e.g. count of scenarios by IRR)
- Investigate this dataset with regression / PCA

In [ ]:
# Example / Starter: Visualise scenarios by IRR, colour by different variables
x = 'IRR'
y = 'avg_gross_margin'

fig = px.scatter(mdf, # output dataframe
                 x=x, 
                 y=y,
                 hover_data=['scenario_id'])

fig.update_xaxes(range=[-0.1,0.2], 
                 title = x,
                 tickformat = '.0%',  
                 gridcolor='lightgrey', # grid lines
                 linecolor='darkgrey', # axis line
                )

fig.update_yaxes(title = 'Average Gross Margin', 
                 tickformat = '.0%', 
                 dtick = 0.01,  
                 gridcolor='lightgrey', 
                 linecolor='darkgrey'
                )

fig.update_traces(marker=dict(size=3, 
                              color = 'rgb(0, 42, 94)'))

fig.update_layout(height = 600, 
                  width = 800, 
                  plot_bgcolor='white'
                 )

fig.show()

#### Conclusion - not a huge relationship between IRR and gross margin - something else might be more important

In [ ]:
# Example / Starter: Visualise scenarios by IRR, colour by different variables
x = 'IRR'
y = 'refinance_interest_rate'

fig = px.scatter(mdf, # output dataframe
                 x=x, 
                 y=y,
                 hover_data=['scenario_id'])

fig.update_xaxes(range=[-0.1,0.2], 
                 title = x,
                 tickformat = '.0%',  
                 gridcolor='lightgrey', # grid lines
                 linecolor='darkgrey', # axis line
                )

fig.update_yaxes(title = 'Refinance Interest Rate', 
                 tickformat = '.0%', 
                 dtick = 0.01,  
                 gridcolor='lightgrey', 
                 linecolor='darkgrey'
                )

fig.update_traces(marker=dict(size=3, 
                              color = 'rgb(0, 42, 94)'))

fig.update_layout(height = 600, 
                  width = 800, 
                  plot_bgcolor='white'
                 )

fig.show()

#### Conclusion - strong negative relationship between IRR and Refinance interest rate
- more important to watch refinancing for this company than margin progression

In [ ]:
x = 'IRR'
y = 'total_capex_as_pct_of_revenue'

fig = px.scatter(mdf, # output dataframe
                 x=x, 
                 y=y,
                 hover_data=['scenario_id'])

fig.update_xaxes(range=[-0.1,0.2], 
                 title = x,
                 tickformat = '.0%',  
                 gridcolor='lightgrey', # grid lines
                 linecolor='darkgrey', # axis line
                )

fig.update_yaxes(range=[0,0.12],
                 title = 'Total Capex as % of Revenue', 
                 tickformat = '.0%', 
                 dtick = 0.01,  
                 gridcolor='lightgrey', 
                 linecolor='darkgrey'
                )

fig.add_vline(x=0)

fig.update_traces(marker=dict(size=3, 
                              color = 'rgb(0, 42, 94)'))

fig.update_layout(height = 600, 
                  width = 800, 
                  plot_bgcolor='white'
                 )

fig.show()

In [ ]:
# Example / Starter: Visualise scenarios by IRR, colour by different variables
x = 'IRR'
y = 'total_capex_as_pct_of_revenue'
fig = px.scatter(mdf, 
                 x=x, 
                 y=y,
                 color='refinance_interest_rate',
                 hover_data=['scenario_id'])

fig.update_xaxes(range=[-0.1,0.2], 
                 title = x,
                 tickformat = '.0%',  
                 gridcolor='lightgrey', # grid lines
                 linecolor='darkgrey', # axis line
                )

fig.update_yaxes(range=[0,0.12],
                 title = 'Total Capex as % of Revenue', 
                 tickformat = '.0%', 
                 dtick = 0.01,  
                 gridcolor='lightgrey', 
                 linecolor='darkgrey'
                )

fig.add_vline(x=0)

fig.update_traces(marker=dict(size=3))

fig.update_layout(height = 600, 
                  width = 800, 
                  plot_bgcolor='white'
                 )
fig.show()

### Multithreading: Use the Whole Computer
- Distribute independent tasks across available threads - e.g, montecarlo simulations!

In [ ]:
# Find out what compute we have available
cpus = cpu_count()
logger.info(f"Detected {cpus} cores on the machine")

def f(x):
    # Simple work function
    return x*x

# Bundle of inputs
number_of_scenarios = range(1000)

# Ensure entry at master process level
if __name__ == '__main__':
    
    # Create a CPU pool
    with Pool(cpus) as p:
        
        results = p.map(f, number_of_scenarios)

In [ ]:
print(len(results))

In [ ]:
results[:5]

- Now let's run our Montecarlo simulation, but multi-threaded

In [ ]:
# Find out what compute we have available
cpus = cpu_count()
logger.info(f"Detected {cpus} cores on the machine")

# Bundle of inputs
number_of_scenarios = 10000

# Ensure entry at master process level
if __name__ == '__main__':
    
    start_time = time.time()
    
    # Create a CPU pool
    with Pool(cpus) as p:
        
        # map the work to the pool, gather the results
        scenarios_pooled = p.map(scenario, [(V, 10)]*number_of_scenarios)
    
    end_time = time.time()
    print(f'Time elapsed: {round(end_time - start_time, 2)}s when distributed across {cpus} CPUs')

### Geospatial Data
- Use geospatial data to sense-check company expansion plans, optimise real estate positioning, etc. 
- Schroders has expanded and has decided to launch a range of cheap Scandinanvian furniture ("SchroKEA"). they have identified 10 potential locations in London for their first store. Which location gives them the greatest addressable market in 20 minutes?

In [ ]:
# Import locations
locations = gpd.read_file('geo_training/locations.geojson')
locations = locations.set_index('id')

# Use TravelTime API service to generate 20-minute isochrone

creds = {'X-Application-Id':os.getenv('X_APPLICATION_ID'),
         'X-Api-Key':os.getenv('X_API_KEY')}
        
travel_time_mins = 20
isos = []
for _id_, location in locations.iterrows():
    
    iso = geo_utils.tt_isochrone(location.geometry, 
                                 travel_time_mins = 20,
                                 creds = creds, # I can't share these!
                                 mode = 'public_transport',
                                 _id_ = str(_id_)
                                )
    
    isos.append(iso)

In [ ]:
iso_gdf = gpd.GeoDataFrame(pd.concat(isos), crs = 4326, geometry = 'geometry')
iso_gdf.index = iso_gdf.index.astype(int)
iso_gdf = iso_gdf.join(locations[['location']]).reset_index()
iso_gdf.to_file('geo_training/isochrones.geojson')

In [21]:
iso_gdf = gpd.read_file('geo_training/isochrones.geojson')

In [22]:
# let's visualise our isochrones for each location, in a different colour each time
colour_map = {1:"rgb(0,42,94)",
             2:"rgb(0,121,109)",
             3:"rgb(234,82,4)",
             4:"rgb(183,25,98)",
             5:"rgb(79,51,152)",
             6:"rgb(0,116,183)",
             7:"rgb(99,197,50)",
             8:"rgb(248,169,8)",
             9:"rgb(223,83,106)",
             10:"rgb(125,55,135)"}

import folium
m = folium.Map(location=[51.5074, -0.1278], 
               zoom_start=12,
               width=1000,
               height=800)  

# Define the style function  
def style_function(feature):  
    
    # Get the value for coloring from the feature properties  
    value = feature['properties']['search_id']
      
    # Get the color based on the value using the color map  
    colour = colour_map.get(value) 
      
    # Return the style properties  
    return {  
        'fillColor': colour,  
        'color': 'black',  
        'weight': 1,  
        'fillOpacity': 0.6,
    }  

folium.GeoJson(iso_gdf, 
               style_function=style_function,
               tooltip=folium.GeoJsonTooltip(fields=['location'])
            ).add_to(m)

display(m)  

In [23]:
# Now, let's import a population dataset so we can see which isochrone affords us the biggest market
with rasterio.open('clipped_GBR_worldpop.tif') as r:
    arr = r.read(1)
    arr[arr<0] = 0
    aff = r.meta['transform']

In [ ]:
# click "raster.PNG" from the files list on the left hand side...

In [44]:
# what does the population data look like "under the hood"?
arr

array([[  0.       ,   0.       ,   7.271873 , ...,   8.592625 ,
          4.4721594,  15.676467 ],
       [  0.       ,   2.4741092,   2.653108 , ...,   3.007857 ,
          2.1524282,   3.0818932],
       [  7.0299067,   6.999458 ,   7.178957 , ...,   1.4412255,
          1.5851892,   1.401784 ],
       ...,
       [ 75.75786  ,  76.0401   , 172.76715  , ...,   5.263302 ,
          8.858702 ,   4.4420295],
       [ 74.39615  ,  72.13014  ,  76.62575  , ...,  15.044149 ,
          2.0365822,   1.207004 ],
       [ 76.76962  ,  75.5393   ,  59.569084 , ...,  14.02362  ,
          1.991469 ,   1.2088212]], dtype=float32)

In [25]:
arr.shape

(154, 346)

In [26]:
aff

Affine(0.0008333333300256081, 0.0, -0.24124998371158846,
       0.0, -0.00083333333002425, 51.57291666154971)

In [27]:
# sample raster for each polygon in iso_gdf
zs = rasterstats.zonal_stats(iso_gdf, arr, affine=aff, stats=['sum'])
iso_gdf['20_min_pop'] = [i['sum'] for i in zs]
iso_gdf = iso_gdf.sort_values(by = '20_min_pop', ascending = False)
iso_gdf

/home/schroders/versioncontrol/re_project_concrete/nvenv/lib/python3.9/site-packages/rasterstats/io.py:328: NodataWarning:

Setting nodata to -999; specify nodata explicitly



,search_id,location,geometry,20_min_pop
7,8,Elephant&Castle,"MULTIPOLYGON (((-0.11584 51.51488, -0.11728 51...",218371.6
8,9,Kilburn,"MULTIPOLYGON (((-0.23408 51.54591, -0.22976 51...",195790.6
4,5,Bank,"MULTIPOLYGON (((-0.14606 51.51308, -0.13886 51...",119317.0
6,7,Shoreditch,"MULTIPOLYGON (((-0.10298 51.51903, -0.10154 51...",99720.9
0,1,Chelsea,"MULTIPOLYGON (((-0.19282 51.47669, -0.19210 51...",96939.1
1,2,Marylebone,"MULTIPOLYGON (((-0.17711 51.52944, -0.17711 51...",94409.0
2,3,Soho,"MULTIPOLYGON (((-0.15765 51.50928, -0.15477 51...",87073.0
3,4,Westminster,"MULTIPOLYGON (((-0.16195 51.48507, -0.16051 51...",58009.1
9,10,Wapping,"MULTIPOLYGON (((-0.08063 51.51240, -0.08063 51...",55027.2
5,6,IsleofDogs,"MULTIPOLYGON (((-0.03814 51.51390, -0.03958 51...",51226.4


- SchrodKEA should set up shop in E&C and Kilburn first, rather than somewhere more central like Westminster - due to a combination of the immediate resident population density + public transport availability

### Alternative Data - Web Scraping
- Gather non-standard data from the internet to augment traditional decision making
- e.g. gather all the property information on this webpage: https://www.rexfordindustrial.com/properties

In [ ]:
url = 'https://www.rexfordindustrial.com/properties'
r = requests.get(url)

In [ ]:
soup = BeautifulSoup(r.content, 'html')

In [ ]:
all_prop_divs = soup.find('div', attrs = {'class':'properties-listing'}).find_all('article')

In [ ]:
prop_collection = []
for a in all_prop_divs:
    p = {'prop_name':a.find('h1').text,
         'location':a.find('h2').text,
         'available_space':a.find_all('h3')[0].text,
         'total_space':a.find_all('h3')[1].text
         }
    prop_collection.append(p)  

We can use this alternative data to track concepts of interest to us...e.g.:
- "What is the fraction of total space, across all warehouse properties, that is currently vacanct?
- "Which property has the highest vacancy rate?
- "Any patterns / identifying factors for the most vacant properties"?

In [ ]:
pdf = pd.DataFrame(prop_collection)

pdf['available_space'].str.split(' ').str[0].str.replace(', ','')

pdf['available_space'] = pd.to_numeric(pdf['available_space'].str.split(' ').str[0].str.replace(',',''), 
                                       errors = 'coerce').fillna(0)

pdf['total_space'] = pd.to_numeric(pdf['total_space'].str.split(' ').str[0].str.replace(',',''), 
                                       errors = 'coerce').fillna(0)

overall_vacancy_rate = pdf['available_space'].sum() / pdf['total_space'].sum()

str(round(overall_vacancy_rate * 100, 2))+' percent'

In [ ]:
pdf['vacancy_rate'] = (pdf['available_space'] / pdf['total_space']) * 100
pdf.sort_values(by = 'vacancy_rate', ascending = False).head(20)

In [ ]:
pdf[pdf['vacancy_rate'] > 80].total_space.mean().round(0)

In [ ]:
pdf[pdf['vacancy_rate'] < 80].total_space.mean().round(0)

In [ ]:
pdf.total_space.mean().round(0)

...The vacant properties are significantly smaller than average for Rexford... are smaller properties harder to lease, or do they have a higher turnover rate?

### Bonus: Machine Learning
- Harness cutting-edge techniques to make predictions for variables of interest
- Better to use Kaggle than look at something I put together! https://www.kaggle.com/code/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets 

### Markowitz Portfolio Optimisation
- Solve optimisation problems, traditional risk-return portfolio optimisation
- turbo charge this with parametric bootstrapping (generate many future scenarios, solve for each!)

In [38]:
import cvxpy as cp
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.datasets import make_spd_matrix
import matplotlib as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Generate Expected Returns
(Entry point for substituting in actual datapoints)

In [53]:
er_names = ['Emerging Markets Equity', 
            'EAFE Equity', 'U.S. Small Cap',
            'U.S. Mid Cap', 
            'U.S. Large Cap', 
            'U.S. High Yield Bonds',
            'Commodities', 
            'U.S. Inv. Grade Corporate Bonds',
            'U.S. Intermediate Treasuries', 
            'TIPS',
            'World ex-U.S. Government Bonds']

random_seed = 2355
np.random.seed(random_seed) # set random state for reproducibility
er_values = np.random.rand(len(er_names)) / 10
er = pd.Series(er_values, index=er_names)
er

Emerging Markets Equity           0.021
EAFE Equity                       0.062
U.S. Small Cap                    0.099
U.S. Mid Cap                      0.028
U.S. Large Cap                    0.096
U.S. High Yield Bonds             0.093
Commodities                       0.097
U.S. Inv. Grade Corporate Bonds   0.065
U.S. Intermediate Treasuries      0.064
TIPS                              0.002
World ex-U.S. Government Bonds    0.061
dtype: float64

### Generate Covariance Matrix
The function 'make_spd_matrix' generates a random covariance matrix 
<i>See: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_spd_matrix.html </i>
(Entry point for replacing with a pre-calculated, real covariance matrix)

In [56]:
cov_dim = er.shape[0]
cov_m = make_spd_matrix(cov_dim, 
                        random_state=random_seed)

# get the diagonal of the matrix and get an array of the indicies in order
order = (-np.diagonal(cov_m)).argsort()

# use that to build a permutation matrix

p = np.eye(cov_dim)[order, :]
# permute the covariance matrix so it's in order
o_cov = p @ cov_m @ p.T

cov = pd.DataFrame(o_cov)
cov.index = er.index
cov.columns = er.index
cov

,Emerging Markets Equity,EAFE Equity,U.S. Small Cap,U.S. Mid Cap,U.S. Large Cap,U.S. High Yield Bonds,Commodities,U.S. Inv. Grade Corporate Bonds,U.S. Intermediate Treasuries,TIPS,World ex-U.S. Government Bonds
Emerging Markets Equity,4.511,3.252,-2.230,2.019,1.374,1.190,0.966,-1.420,-1.087,1.158,-0.347
EAFE Equity,3.252,3.205,-1.692,1.577,1.130,0.937,0.742,-1.045,-0.986,0.849,-0.204
U.S. Small Cap,-2.230,-1.692,1.666,-1.158,-0.887,-0.737,-0.551,0.814,0.583,-0.546,0.304
U.S. Mid Cap,2.019,1.577,-1.158,1.482,0.579,0.609,0.579,-0.622,-0.523,0.494,-0.183
U.S. Large Cap,1.374,1.130,-0.887,0.579,1.073,0.399,0.268,-0.488,-0.290,0.459,-0.065
U.S. High Yield Bonds,1.190,0.937,-0.737,0.609,0.399,1.000,0.221,-0.493,-0.344,0.341,-0.156
Commodities,0.966,0.742,-0.551,0.579,0.268,0.221,0.937,-0.360,-0.237,0.166,-0.208
U.S. Inv. Grade Corporate Bonds,-1.420,-1.045,0.814,-0.622,-0.488,-0.493,-0.360,0.925,0.360,-0.412,0.109
U.S. Intermediate Treasuries,-1.087,-0.986,0.583,-0.523,-0.290,-0.344,-0.237,0.360,0.868,-0.349,0.026
TIPS,1.158,0.849,-0.546,0.494,0.459,0.341,0.166,-0.412,-0.349,0.761,-0.013


In [57]:
# we can also visualise this very cheaply
px.imshow(o_cov)

In [121]:
def optimisation_simple(Return, risk, gamma, w, optim_params):
    
    """
    simple optimisation - directly inject gamma values into the problem from a risk-aversion logspace
    """
    
    # define problem, params - standard Markowitz portfolio optimisation 
    prob = cp.Problem(cp.Maximize(Return - gamma * risk), # maximise return for a given level of risk
                      [cp.sum(w) == 1, # constaint: ensure our portfolio weights sum to 1
                       w >= 0.0 # constaint:ensure every weight is greater or equal to 0 - no shorting
                      ] 
                     )
    
    risk_data = np.zeros(optim_params['samples']) # setup container
    Return_data = np.zeros(optim_params['samples']) # setup container
    weights = [] # setup container
    
    # define our levels of risk aversion
    gamma_vals = np.logspace(-2, 1, num=optim_params['samples'])
    
    for i in range(optim_params['samples']):
        gamma.value = gamma_vals[i]
        prob.solve()
        risk_data[i] = cp.sqrt(risk).value
        Return_data[i] = Return.value
        weights.append(np.squeeze(np.asarray(w.value)))
        
    return risk_data, Return_data, weights

In [133]:
def run_simulations(er: pd.Series, 
                    cov: pd.DataFrame, 
                    number_of_sims: int, 
                    optim_params: dict):
    
    """
    er: pandas Series - describing expected returns 
    cov: pandas Dataframe - describing covariance matrix of historical returns
    number_of_sims: int - number of simulations to perform
    optim_params - optimisation parameters, passed through to optimal_portfolio 
    rs_explore_type - simulation method. We prefer risk targeting, set as default
    """
    
    # Create a monthly datetime index
    dates = pd.date_range(start='2010-08-31', 
                          periods=120, 
                          freq='M')

    # Generate 10 years of monthly data by sampling ("parametric bootstraping")
    # these will be carried forward for all of the simulations
    data = []
    for i in range(0, number_of_sims):
        if i+1 % 5 == 0:
            print(f"Generating data for sim {i+1}")
        data.append(
            pd.DataFrame(
                columns = cov.columns,
                index = dates,
                data = np.random.multivariate_normal(er.values, cov.values, 120)
            ))
        
    # Store values from Optimisation 
    stdev_col = [None for i in range(number_of_sims)]
    exp_ret_col = [None for i in range(number_of_sims)]
    weight_col = [None for i in range(number_of_sims)]
    
    # For each forward return simulation, find the optimal portfolio
    for i in range(number_of_sims):
        if i+1 % 5 == 0:
            print(f"Building portfolios for sim {i+1}")
            
        this_simulation_returns = data[i]
        
        # Set Problem Parameters in this scenario
        n = len(this_simulation_returns.columns) # number of securities
        w = cp.Variable(n) # setup a variable, w, of array size n
        mu = this_simulation_returns.mean() * 12 # simple annualisation of returns
        Sigma = this_simulation_returns.cov() * 12 # simple covariance of returns
        gamma = cp.Parameter(nonneg=True) # risk aversion parameter
        ret = mu.values.T @ w # return is a function of mean returns x portfolio weights
        risk = cp.quad_form(w, Sigma.values) # risk is a quadratic form variable
        
        # Optimize over every simulation
        stdev, exp_ret, weights = optimisation_simple( Return = ret, 
                                                            risk = risk, 
                                                            gamma = gamma, 
                                                            w = w, 
                                                            optim_params = optim_params)
               
        stdev_col[i] = stdev
        exp_ret_col[i] = exp_ret
        weight_col[i] = pd.DataFrame(data = weights, 
                                     columns = this_simulation_returns.columns)
            
    return weight_col, stdev_col, exp_ret_col

In [134]:
OPTIM_PARAMS = {'samples':200,
                'sparse_samples':int(200 / 5),
                'jumpsize':0.05}

weight_col, stdev_col, exp_ret_col = run_simulations(er = er, 
                                                     cov = cov, 
                                                     number_of_sims = 10, 
                                                     optim_params = OPTIM_PARAMS)

### Visualise / Plot Efficient Frontiers

In [140]:
fig = go.Figure()
for i, result in enumerate(zip(stdev_col, exp_ret_col)):
    risk = result[0]
    rtn = result[1]
    fig.add_trace(go.Scatter(x=risk, 
                             y=rtn, 
                             name=f'Simulation {i+1}'), 
                 )
    
xmin, xmax = (np.floor(min([min(sub.x) for sub in fig.data])), 
              np.ceil(max([max(sub.x) for sub in fig.data])) )
ymin, ymax = (np.floor(min([min(sub.y) for sub in fig.data])), 
              np.ceil(max([max(sub.y) for sub in fig.data])) )

fig.update_layout(
    xaxis = {
        "range": [xmin, xmax],
        "title": 'Risk'
    },
    yaxis = {
        "range": [ymin, ymax],
        "title":'Return',
    },
    title = "Markowitz Portfolio Optimisation - An Efficient Frontier for Every Future!",
    height=500,
    template="plotly_white"
)

fig.show()